# Infer RNA topics

In [ ]:
%load_ext autoreload
%autoreload 2
    
import os
os.environ["RAY_TQDM_PATCH_PRINT"]="0"
from umap import *
import numpy as np
import scprinter as scp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import os
import pickle
import torch
import matplotlib as mpl
import scanpy as sc
import anndata as ad
from scmallet import *
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
atac_adata = scp.pp.make_peak_matrix(printer, f'{work_dir}/{peak}', 300)

In [25]:
atac_adata.write(f'{work_dir}/cellxpeak.h5ad')

In [9]:
atac_adata = ad.read_h5ad(f'{work_dir}/cellxpeak.h5ad')

In [10]:
rna_adata = sc.read_mtx('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10-RNA.raw_UMI.merged.converted_barcodes.mtx.gz')
rna_adata = ad.AnnData(X = rna_adata.X.T)
bc = [line.strip() for line in open('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10-RNA.raw_UMI.merged.converted_barcodes.barcodes.list', 'r').readlines()]
rna_adata.obs.index = bc
bc = [line.strip() for line in open('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10-RNA.raw_UMI.merged.converted_barcodes.genes.list', 'r').readlines()]
rna_adata.var.index = bc

In [11]:
selected_gene = [line.strip() for line in open('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10-RNA.thresh_0.5.genes.list' , 'r').readlines()]
rna_adata = rna_adata[:, selected_gene].copy()

In [12]:
rna_adata

AnnData object with n_obs × n_vars = 111877 × 13220

In [13]:
stem_bcs = [xx.strip() for xx in open('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10.shared_stem_barcodes.list').readlines()]

In [15]:
meta_cell = pd.read_csv('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10-ATAC.qc_stats.singlet.txt', sep='\t')
shared = list(set(meta_cell['barcode']) &  set(atac_adata.obs.index)  & set(rna_adata.obs.index))

In [16]:
atac_adata = atac_adata[shared].copy()
rna_adata = rna_adata[shared].copy()

In [17]:
atac_adata.X.data = np.ones_like(atac_adata.X.data)

In [18]:
atac_adata

AnnData object with n_obs × n_vars = 52540 × 127624

In [19]:
rna_adata

AnnData object with n_obs × n_vars = 52540 × 13220

In [28]:
model = Mallet('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/atac_cell_topics')
for topic_num in [90][::-1]:
    model.fit(topic_num,
              atac_adata,
              cpu_per_task=32,mem_gb=512,
              iterations=500)

region_topics = model.get_region_topics(90)
cell_topics = model.get_cell_topics(90)
region_topics.to_csv(f'/data/rzhang/data_15/colitisDSS/tissue/joint_topic/topic90_region_topics_ATAC.txt.gz', sep='\t')
cell_topics.to_csv(f'/data/rzhang/data_15/colitisDSS/tissue/joint_topic/topic90_cell_topics_ATAC.txt.gz', sep='\t')


In [29]:
model = Mallet('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/rna_cell_topics')
for topic_num in [90][::-1]:
    model.fit(topic_num,
              rna_adata,
              cpu_per_task=32,mem_gb=512,
              iterations=500)
xx = model.get_cell_topics(90)

region_topics = model.get_region_topics(90)
cell_topics = model.get_cell_topics(90)

region_topics.to_csv(f'/data/rzhang/data_15/colitisDSS/tissue/joint_topic/topic90_region_topics_RNA.txt.gz', sep='\t')
cell_topics.to_csv(f'/data/rzhang/data_15/colitisDSS/tissue/joint_topic/topic90_cell_topics_RNA.txt.gz', sep='\t')